In [ ]:
##exxu5fm13b8f5mfpneyo

In [1]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata

import vertexai
import vertexai.generative_models as genai
from vertexai.generative_models import GenerativeModel, Part, GenerationConfig
import json
import time
import os
import pandas as pd
import numpy as np 
from dotenv import load_dotenv
load_dotenv(verbose=True)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../input/mtec-unstructured-data-390708-c264fdc6ca7a.json"
vertexai.init()
# モデルの準備
gemini = genai.GenerativeModel("models/gemini-1.5-flash")
from IPython.display import display, HTML
from pydantic import BaseModel, Field
from openai import AzureOpenAI
engine='gpt-4o-mini'
gpt = AzureOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.environ.get("OPENAI_ENDPOINT"),
)

In [2]:
def load_data(path):
    with open(path, 'rb') as f:
        data = pd.DataFrame().from_dict(json.load(f)).T.reset_index(names="question_number")
        f.close()
    return data

In [3]:
df_train = load_data("../train/questions.json")
df_train['flag'] = 'train'
df_eval = load_data("../valid/questions.json")
df_eval['flag'] = 'eval'
df_test = load_data("../test/questions.json")
df_test['flag'] = 'test'
df = pd.concat([df_train, df_eval, df_test], axis=0)

In [50]:
class TQAtast():
    def __init__(self, question, table_id, flag):
        self.question = question
        self.table_id = table_id
        self.flag = flag
        self.doc_id = table_id.split("-")[0]
        self.file = table_id.split("-")[1]
        self.file_path = f'../{self.flag}/reports/{self.doc_id}/{self.file}.html'
    
    def get_table(self):
        with open(self.file_path, 'r') as f:
            html = f.read()
            f.close()
        soup = bs(html, 'html.parser')
        table = soup.find('table', attrs={'table-id': self.table_id})
        return table
    
    def restructuring_table(self,):
        table = self.get_table()
        rows = table.find_all('tr')
        table_str = ['<table>']

        inner_tables = table.find_all('table')
        for inner_table in inner_tables:
            # Get the text inside the inner table
            inner_text = unicodedata.normalize('NFKC', inner_table.get_text()).replace('\n', '').replace(' ', '')
            # Replace the inner table with its text
            inner_table.replace_with(inner_text)
            inner_table.decompose()

        for row in rows:
            cells = row.find_all(['td', 'th', 'table'])
            table_str.append('<tr>')
            row = []
            for cell in cells:
                text = unicodedata.normalize('NFKC', cell.get_text()).replace('\n', '').replace(' ', '')
                if cell.get('colspan') and cell.get('rowspan'):
                    tmp = f'<td colspan="{cell.get("colspan")}" rowspan="{cell.get("rowspan")}" cell-id={cell.get("cell-id")}>{text}</td>'
                elif cell.get('colspan'):
                    tmp = f'<td colspan="{cell.get("colspan")}" cell-id={cell.get("cell-id")}>{text}</td>'
                elif cell.get('rowspan'):
                    tmp = f'<td rowspan="{cell.get("rowspan")}" cell-id={cell.get("cell-id")}>{text}</td>'
                else:
                    tmp = f'<td cell-id={cell.get("cell-id")}>{text}</td>'
                table_str.append(tmp)
            table_str.append('</tr>')
        table_str.append('</table>')
        return '\n'.join(table_str)
    
    def edit_question(self, ):
        question = unicodedata.normalize('NFKC', self.question)
        if re.search(r"20(\d+)年", question):
            if re.search(r"20(\d+)年(.*)", self.question).group(1) == '24':
                question = re.sub(r"20(\d+)年", "2024年(当期)",question)
            
            elif re.search(r"20(\d+)年(.*)", self.question).group(1) == '23':
                question = re.sub(r"20(\d+)年", "2023年(前期)",question)
            
            elif re.search(r"20(\d+)年(.*)", self.question).group(1) == '22':
                question = re.sub(r"20(\d+)年", "2022年(2期前)",question)
            
            elif re.search(r"20(\d+)年(.*)", self.question).group(1) == '21':
                question = re.sub(r"20(\d+)年", "2021年(3期前)",question)
            
            elif re.search(r"20(\d+)年(.*)", self.question).group(1) == '20':
                question = re.sub(r"20(\d+)年", "2020年(4期前)",question)

        if re.search(r"20(\d+)年(.*)", question):
            question = re.search(r"20(\d+)年(.*)", question).group(0)
        
        
        if re.search(r"「(.*)」", question):
            inner_text = re.search(r"「(.*)」", question).group(1)
            inner_text = re.sub(r"\(.*\)", "", inner_text)
            split_text = re.split(r'-|、',inner_text)[::-1]
            joined_text = 'の'.join(split_text)
            question = re.sub(r"「(.*)」", joined_text, question)
        
        return question.replace('は?', 'の値は?')

    def get_cell(self, model):
        table_text = self.restructuring_table()
        question = self.edit_question() 
        print(question)
        class Cell_id(BaseModel):
            cell_id: str = Field(description=f"セルID。{self.table_id}-r1c1のような形式で記載される",)
            value: str = Field(description="単位も含めた質問の回答。抽出できない場合は-としてください。",)
            unit: str = Field(description="単位。抽出できない場合は-としてください。",)

        if model == 'gemini':
            prompt = f"あなたは2024年の有価証券報告書から情報抽出タスクを任されました。質問に対する回答をHTMLで記述された表を参考に回答してください。##質問:{question} \n##表:{table_text}"
            response = gemini.generate_content(
                [prompt],
                generation_config=GenerationConfig(response_mime_type="application/json", response_schema=Cell_id.model_json_schema())
            )
            
            return json.loads(response.text)

        elif model == 'gpt':
            message = [
                {"role": "system","content":[{"type": "text", "text": "あなたは2024年の有価証券報告書から情報抽出タスクを任されました"}]},
                {"role": "user","content":[{"type": "text", "text":f"質問に対する回答をHTMLで記述された表を参考に回答してください。##質問:{question} \n##表:{table_text}"}]}
            ]
    
            OUTPUT_FUNCTION = {
                "name": "output",
                "parameters":Cell_id.model_json_schema(),
            }
            
            response = gpt.beta.chat.completions.parse(
                model=engine,
                messages=message,
                functions=[OUTPUT_FUNCTION],
                function_call={"name": OUTPUT_FUNCTION["name"]},
                max_tokens=2000,
                temperature=0.,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
            )
            
            return json.loads(response.choices[0].message.function_call.arguments)


In [51]:
row = df.iloc[14]
task = TQAtast(row['question'], row['table_id'], row['flag'])
display(HTML(task.restructuring_table()))
print(row['question'])
print(task.get_cell('gpt'))
print(row['cell_id'], row['value'])

,,(単位:百万円)
,前事業年度(2022年12月31日),当事業年度(2023年12月31日)
負債の部,,
流動負債,,
支払手形,"※1,※31,383","※1,※325"
支払電子手形,"※1,※3227,820","※1,※3110,338"
買掛金,"※182,412","※180,188"
コマーシャル・ペーパー,"25,000",-
短期借入金,"30,000",-
未払金,"※124,637","※121,074"
未払法人税等,"5,902","13,788"


株式会社　クボタの2023年時点の個別決算における「土地圧縮積立金」は？
2023年(前期)時点の個別決算における土地圧縮積立金の値は?
{'cell_id': 'S100T32K-0105020-tab305-r37c3', 'value': '171', 'unit': '百万円'}
S100T32K-0105020-tab305-r37c3 171000000


In [23]:
#extracted_data = {}

count = 100
correct = 0
for index, row in df_train[:count].iterrows():
    task = TQAtast(row['question'], row['table_id'], row['flag'])
    answer = task.get_cell(model='gpt')
    if answer['cell_id'] == row['cell_id']:
        correct += 1
    else:
        print(index, row['question'])
        print(row['cell_id'], row['value'])
        print(answer)
        print()


0 株式会社しずおかフィナンシャルグループの2024年の個別決算における「営業外収益」は？
S100TLFZ-0105320-tab219-r14c4 0
{'cell_id': 'S100TLFZ-0105320-tab219-r11c2', 'cell_text': '営業外収益', 'unit': ''}

5 株式会社ディー・エヌ・エーの2024年時点のSatoAtsukoMemberにおける「任期、役員の状況（取締役（及び監査役））」は？
S100TQ8O-0104010-tab63-r2c25 (注)４
{'cell_id': 'S100TQ8O-0104010-tab63-r2c5', 'cell_text': '―', 'unit': ''}

6 株式会社ユー・エス・エスの2024年時点のOrdinaryShareMemberにおける「内容、発行済株式、株式の総数等」は？
S100TUHI-0104010-tab43-r2c5 単元株式数100株
{'cell_id': 'S100TUHI-0104010-tab43-r2c2', 'cell_text': '257,000,000', 'unit': '株'}

9 株式会社レゾナック・ホールディングスの2023年時点の個別決算における「投資その他の資産」は？
S100T4LL-0105020-tab201-r37c3 1254692000000
{'cell_id': 'S100T4LL-0105020-tab201-r31c3', 'cell_text': '-', 'unit': ''}

13 株式会社カネカの2024年の連結決算のNutritionSolutionsUnitReportableSegmentMemberにおける「売上高」は？
S100TV9M-0105110-tab180-r7c5 187229000000
{'cell_id': 'S100TV9M-0105110-tab180-r4c5', 'cell_text': '187,182', 'unit': '百万円'}

14 株式会社　クボタの2023年時点の個別決算における「土地圧縮積立金」は？
S100T32K-0105020-tab305-r37c3 171000000
{'cell_id': 'S100T32K

In [24]:
print(correct/count)

0.6


In [9]:
#extracted_data = json.loads(open("../test/extracted_data.json", "r").read())
answersheet = json.loads(open("../test/answersheet.json", "r").read())
tableqa = json.loads(open("../test/tableqa.json", "r").read())

In [10]:
def normalize(cell_text, unit):
    cell_text = unicodedata.normalize('NFKC', cell_text).replace(',', '')
    if '-' == cell_text:
        return '-'
    elif '百万' in unit:
        return cell_text + '000000'
    elif '千' in unit:
        return cell_text + '000'
    elif '%' in unit:
        return cell_text+'%'
    else:
        return cell_text

In [11]:
for key in answersheet.keys():
    answersheet[key]['cell_id'] = extracted_data[key]['cell_id']
    answersheet[key]['value'] =normalize(extracted_data[key]['cell_text'], extracted_data[key]['unit'])

In [13]:
with open("../test/answersheet2.json", "w") as f:
    json.dump(answersheet, f, indent=4, ensure_ascii=False)

In [59]:
tableqa = json.loads(open("../test/tableqa.json", "r").read())

In [60]:
count = 0
id_correct = 0
value_correct = 0
for key in tableqa.keys():
    if tableqa[key]['cell_id'] == answersheet[key]['cell_id']:
        id_correct += 1
    else:
        print(key, tableqa[key]['cell_id'], answersheet[key]['cell_id'])
    

    if tableqa[key]['value'] == answersheet[key]['value']:
        value_correct += 1
    else:
        print(key, tableqa[key]['value'], answersheet[key]['value'])
    count += 1

question_tqa_test2 S100TI2A-0105010-tab91-r2c2 S100TI2A-0105010-tab91-r16c2
question_tqa_test3 S100SC8R-0104010-tab65-r5c2 S100SC8R-0104010-tab65-r5c1
question_tqa_test3 39000000 社外取締役
question_tqa_test5 100000000 100
question_tqa_test9 S100TUZV-0103010-tab24-r6c2 S100TUZV-0103010-tab24-r2c2
question_tqa_test9 32135000000 12013000000
question_tqa_test10 S100SDFR-0105320-tab198-r5c3 S100SDFR-0105320-tab198-r18c3
question_tqa_test11 S100TO17-0105310-tab185-r33c6 S100TO17-0105310-tab185-r20c7
question_tqa_test19 S100TOTW-0101010-tab13-r2c5 S100TOTW-0101010-tab13-r2c4
question_tqa_test21 S100TUIH-0104010-tab70-r41c2 S100TUIH-0104010-tab70-r25c2
question_tqa_test25 S100T3GA-0101010-tab3-r4c6 S100T3GA-0101010-tab3-r4c5
question_tqa_test25 20233000000 18622000000
question_tqa_test30 S100SC8R-0104010-tab57-r12c2 S100SC8R-0104010-tab57-r2c5
question_tqa_test30 (注)1 1991年4月
question_tqa_test33 S100TW7S-0101010-tab3-r23c7 S100TW7S-0101010-tab3-r13c5
question_tqa_test33 -96.77 55.11
question_tqa_t

In [20]:
row = df.query('question_number == "question_tqa_test3"').iloc[0]
task = TQAtast(row['question'], row['table_id'], row['flag'])
print(row['question'])
print(task.restructuring_table())

マニー株式会社の2023年のOutsideDirectorsMemberにおける「譲渡制限付株式報酬、役員区分ごとの報酬等」は？
<tr>
<td S100SC8R-0104010-tab65-r1c1>役員区分</td>
<td S100SC8R-0104010-tab65-r1c2>報酬等の総額(百万円)</td>
<td S100SC8R-0104010-tab65-r1c3>報酬等の種類別の総額(百万円)</td>
<td S100SC8R-0104010-tab65-r1c4>対象となる役員の員数</td>
</tr>
<tr>
<td S100SC8R-0104010-tab65-r2c1>基本報酬</td>
<td S100SC8R-0104010-tab65-r2c2>業績連動報酬</td>
<td S100SC8R-0104010-tab65-r2c3>譲渡制限付株式報酬</td>
<td S100SC8R-0104010-tab65-r2c4>左記のうち、非金銭報酬等</td>
</tr>
<tr>
<td S100SC8R-0104010-tab65-r3c1>取締役(社外取締役を除く)</td>
<td S100SC8R-0104010-tab65-r3c2>9</td>
<td S100SC8R-0104010-tab65-r3c3>9</td>
<td S100SC8R-0104010-tab65-r3c4>-</td>
<td S100SC8R-0104010-tab65-r3c5>-</td>
<td S100SC8R-0104010-tab65-r3c6>-</td>
<td S100SC8R-0104010-tab65-r3c7>1名</td>
</tr>
<tr>
<td S100SC8R-0104010-tab65-r4c1>執行役</td>
<td S100SC8R-0104010-tab65-r4c2>272</td>
<td S100SC8R-0104010-tab65-r4c3>169</td>
<td S100SC8R-0104010-tab65-r4c4>75</td>
<td S100SC8R-0104010-tab65-r4c5>27</td>
<td S100SC8R-0104010-tab65-r4c6>2

In [62]:
2429/4265

0.5695193434935522